# clean

In [24]:
# import

import csv
import os
import re
from pathlib import Path

import sys
import os

# Chemin du dossier parent (où se trouve Fonction_cleanning)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(parent_dir)


from Fonction_cleanning import fonc_clean_MCLG as clean

from Fonction_cleanning import map as map



In [25]:
# --- Fonction utilitaire ---
def ensure_output_dir(path="./output/"):
    Path(path).mkdir(parents=True,exist_ok=True)


In [26]:
def ensure_output_csv(path="./output.csv"):
    """Create the output CSV file if it does not already exist."""
    Path(path).parent.mkdir(parents=True,exist_ok=True)
    Path(path).touch(exist_ok=True)

In [27]:
# --- Nettoyage : suppression des lignes vides ---
def remove_empty_rows(input_csv:str, output_csv:str, column_name:str):
    ensure_output_dir()
    with open(input_csv, 'r', newline='', encoding='utf-8') as infile, \
         open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if column_name in row and row[column_name].strip():
                writer.writerow(row)


In [28]:
# --- Filtrage des lignes selon une valeur ---

def filter(input_csv:str, output_csv:str, column_name:str, valeurs: list):
    """
    Parcourt un CSV et vide les cellules de la colonne 'column_name'
    si leur valeur figure dans 'valeurs' (insensible à la casse et aux espaces).
    Les lignes ne sont pas supprimées.
    """
    ensure_output_dir()
    

    with open(input_csv, 'r', newline='', encoding='utf-8') as infile, \
         open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()

        valeurs_normalisées = [v.strip().lower() for v in valeurs]

        for row in reader:
            if column_name in row:
                cell_value = row[column_name].strip().lower()
                if cell_value in valeurs_normalisées:
                    # On vide la cellule si la valeur est à exclure
                    row[column_name] = ''
            writer.writerow(row)


In [29]:
# --- Suppression de colonnes ---
def remove_columns(input_csv:str, output_csv:str, columns_to_remove:dict):
    ensure_output_dir()
    
    with open(input_csv, 'r', newline='', encoding='utf-8') as infile, \
         open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        columns_to_keep = [c for c in reader.fieldnames if c not in columns_to_remove]
        writer = csv.DictWriter(outfile, fieldnames=columns_to_keep)
        writer.writeheader()
        for row in reader:
            writer.writerow({c: row[c] for c in columns_to_keep})


In [30]:
# --- Conservation de colonnes spécifiques ---
def keep_columns(input_csv:str, output_csv:str, columns_to_keep:dict):
    ensure_output_dir()
    ()
    with open(input_csv, 'r', newline='', encoding='utf-8') as infile, \
         open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        valid_columns = [c for c in columns_to_keep if c in reader.fieldnames]
        writer = csv.DictWriter(outfile, fieldnames=valid_columns)
        writer.writeheader()

        single_column = len(valid_columns) == 1
        col_name = valid_columns[0] if single_column else None

        for row in reader:
            if single_column:
                
                value = row[col_name].strip()
                
                
                writer.writerow({col_name: value})
               
            else:
                writer.writerow({c: row[c] for c in valid_columns})




In [31]:
def explode_single_column_csv_to_ids(input_csv, output_csv, mapping_dict=None, delimiter=','):
    """
    Transforme un CSV à une colonne en un CSV plat (user, id_valeur).
    - mapping_dict (optionnel) : dict {valeur: id}
    - Si une valeur n'existe pas dans le dict, un nouvel id est créé automatiquement.
    - Retourne le dictionnaire final mis à jour.
    """
    Path(output_csv).parent.mkdir(parents=True, exist_ok=True)

    # Initialiser le dictionnaire si absent
    if mapping_dict is None:
        dict = {}
    else :
        dict = mapping_dict
    # Calculer le prochain ID disponible
    next_id = max(dict.values(), default=0) + 1

    with open(input_csv, 'r', newline='', encoding='utf-8-sig') as infile:
        reader = csv.DictReader(infile)

        # Normaliser le nom de la seule colonne
        fieldnames = [f.strip().strip('"').strip("'") for f in reader.fieldnames]
        if len(fieldnames) != 1:
            raise ValueError(f"Le fichier doit contenir une seule colonne, trouvé : {fieldnames}")

        column_name = fieldnames[0]
        print(f"[DEBUG] Colonne détectée : {column_name!r}")

        with open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
            fieldnames_out = ['id_user', f'id_{column_name}']
            writer = csv.DictWriter(outfile, fieldnames=fieldnames_out)
            writer.writeheader()

            for user_id, row in enumerate(reader, start=1):
                # Trouver la clé réelle dans le dictionnaire
                for key in row.keys():
                    if key.strip().strip('"').strip("'") == column_name:
                        cell_value = row[key]
                        break
                else:
                    raise KeyError(f"Impossible de trouver la colonne {column_name} dans la ligne {row}")

                if not cell_value:
                    continue

                # Nettoyer la cellule
                cell_value = cell_value.strip().strip('"').strip("'")
                items = [item.strip() for item in cell_value.split(delimiter) if item.strip()]

                for item in items:
                    # Vérifier si la valeur a déjà un ID
                    if item not in dict:
                        dict[item] = next_id
                        next_id += 1
                        print(f"[DEBUG] Nouvelle valeur mappée : {item!r} -> ID {dict[item]}")
                    writer.writerow({'id_user': user_id, f'id_{column_name}': dict[item]})
                # print(f'//id_{column_name}//')

    return dict

In [32]:
def safe_filename(name: str) -> str:
    """Nettoie un nom de fichier en remplaçant les caractères interdits."""
    # Supprime ou remplace les caractères interdits
    name = name.strip().replace(' ', '_')
    # Enlève tous les caractères non alphanumériques sauf `_` et `-`
    name = re.sub(r'[^A-Za-z0-9_\-]', '', name)
    return name

In [33]:
import os
from pathlib import Path

def donne_table_association(input: str, colonne: str,
                            mapping_dict_: dict = None,
                            only_map: bool = False):
    """
    Génère les tables d'association et renvoie un mapping final
    pour la colonne choisie dans un CSV.
    """

    col_nom = safe_filename(str(colonne))

    # Sécurisation des dictionnaires
    mapping_full_input = mapping_dict_.copy() if mapping_dict_ else {}
    mapping_final_input = mapping_dict_.copy() if mapping_dict_ else {}

    # print("Clés actuelles du mapping :", mapping_final_input.keys())

    # Fichiers temporaires
    tmp_file = Path(f"./relation/{col_nom}_temp.csv")
    filtered_file = Path(f"./relation/{col_nom}.csv")
    id_file = Path(f"./relation/{col_nom}_id.csv")
    id_filtered_file = Path(f"./relation/{col_nom}_id_final.csv")

    nom_colonne_id = f"id_{colonne}"
    nom_colonne_id = nom_colonne_id.strip().strip('"').strip("'")
    # Étape 1 : extraction de la colonne
    keep_columns(input, tmp_file, [colonne])

    # Étape 2 : filtrage des valeurs non désirées
    filter(tmp_file, filtered_file, colonne, ["Ne souhaite pas répondre"])

    # Étape 3 : on supprime le fichier temporaire si présent
    if tmp_file.exists():
        tmp_file.unlink()

    # Étape 4 : génération du mapping (ID ↔ valeurs)
    mapping_full = explode_single_column_csv_to_ids(
        filtered_file,
        id_file,
        mapping_full_input
    )

    # Mode only_map → filtrage sur les clés du dictionnaire fourni
    if only_map:
        # print("[INFO] Mode only_map activé")

        allowed_keys = set(mapping_final_input.keys())
        # print("Clés autorisées :", allowed_keys)

        # valeurs à retirer
        to_remove = [key for key in mapping_full.keys() if key not in allowed_keys]
        # print("Valeurs supprimées :", to_remove)
        to_remove_id =[str(mapping_full[key]) for key in to_remove]
        print(to_remove_id)
        # Filtrage du fichier d’IDs
        print(f"//{nom_colonne_id}//")
        filter(id_file, id_filtered_file, nom_colonne_id, to_remove_id)

        # Suppression du fichier intermédiaire
        # if id_file.exists():
        #     id_file.unlink()

    return mapping_full


# init

In [34]:
ensure_output_dir("./output/")
ensure_output_dir("./table/")
ensure_output_dir("./relation/")


# Test

In [35]:
# Colonnes à supprimer (fausses questions ou inutiles)
l_fausse_question = [
    "Souhaitez vous accepter le traitement de vos données ?",
    "Influence culturelle",
    " [Ligne 1]",
    "Colonne 12",
    "Reconnaissez-vous avoir lu et accepté ces conditions.",
    "Colonne 20",
    "dzdzd",
    "Souhaitez-vous envoyer le questionnaire ?",
    "Colonne 16",
    "Colonne 17",
    "Colonne 15"
]
ensure_output_csv()
# Suppression de colonnes inutiles
remove_columns('./input.csv', './output/output.csv', l_fausse_question)


In [36]:
# Extraction des questions ouvertes
l_question_ouverte = [
    "Quel est votre morceau préféré en ce moment ? ",
    "Quel est votre artiste ou groupe préféré ?"
]
keep_columns('./input.csv', './output/ouvert.csv', l_question_ouverte)

In [37]:

# Extraction des choix multiples
l_choix_multiple = [
    "Quels genres écoutez-vous le plus ?",
    "Période(s) écoutée(s) ",
    "Langue(s) écoutée(s)",
    "Sur quelle plateforme écoutez vous de la musique ?",
    "Sous quel(s) mood(s) écoutez-vous de la musique ?",
    "Dans quel contexte écoutez-vous vos musiques ?"
]
keep_columns('./input.csv', './output/multiple.csv', l_choix_multiple)



In [38]:
# --- TON CAS FINAL --- #

l_choix_multiple_periode = ["Période(s) écoutée(s) "]

# Étape 1 : on extrait uniquement cette colonne
keep_columns('./input.csv','./output/multiple_periode_temp.csv',l_choix_multiple_periode)

# Étape 2 : on filtre pour enlever les "Ne souhaite pas répondre"
filter(
    './output/multiple_periode_temp.csv', './output/multiple_periode.csv', "Période(s) écoutée(s) ",["Ne souhaite pas répondre"]
)

# Étape 3 : (optionnel) supprimer le fichier temporaire
os.remove('./output/multiple_periode_temp.csv')


mapping_full = explode_single_column_csv_to_ids(
    './output/multiple_periode.csv',
    './output/multiple_periode_flat.csv'
)

mapping = {
    "2020’s": 1,
    "2010’s": 2
}

mapping_final = explode_single_column_csv_to_ids(
    './output/multiple_periode.csv',
    './output/multiple_periode_flat_ids.csv',
    mapping_dict=mapping
)

# print(mapping_final)

à_enlever = ['7','8']



# Étape 2 : on filtre pour enlever les "Ne souhaite pas répondre,'musique classique'"
filter('./output/multiple_periode_flat_ids.csv', './output/multiple_periode_flat_ids_final.csv', 'id_Période(s) écoutée(s)',à_enlever)

# Étape 3 : (optionnel) supprimer le fichier temporaire
os.remove('./output/multiple_periode_flat_ids.csv')



[DEBUG] Colonne détectée : 'Période(s) écoutée(s)'
[DEBUG] Nouvelle valeur mappée : '2020’s' -> ID 1
[DEBUG] Nouvelle valeur mappée : '2010’s' -> ID 2
[DEBUG] Nouvelle valeur mappée : '2000’s' -> ID 3
[DEBUG] Nouvelle valeur mappée : '1990’s' -> ID 4
[DEBUG] Nouvelle valeur mappée : '1980’s' -> ID 5
[DEBUG] Nouvelle valeur mappée : '1970’s ou avant' -> ID 6
[DEBUG] Nouvelle valeur mappée : 'musique classique' -> ID 7
[DEBUG] Nouvelle valeur mappée : 'Ne souhaite pas répondre' -> ID 8
[DEBUG] Colonne détectée : 'Période(s) écoutée(s)'
[DEBUG] Nouvelle valeur mappée : '2000’s' -> ID 3
[DEBUG] Nouvelle valeur mappée : '1990’s' -> ID 4
[DEBUG] Nouvelle valeur mappée : '1980’s' -> ID 5
[DEBUG] Nouvelle valeur mappée : '1970’s ou avant' -> ID 6
[DEBUG] Nouvelle valeur mappée : 'musique classique' -> ID 7
[DEBUG] Nouvelle valeur mappée : 'Ne souhaite pas répondre' -> ID 8


In [39]:
donne_table_association(input='./input.csv',colonne="Période(s) écoutée(s) ")


[DEBUG] Colonne détectée : 'Période(s) écoutée(s)'
[DEBUG] Nouvelle valeur mappée : '2020’s' -> ID 1
[DEBUG] Nouvelle valeur mappée : '2010’s' -> ID 2
[DEBUG] Nouvelle valeur mappée : '2000’s' -> ID 3
[DEBUG] Nouvelle valeur mappée : '1990’s' -> ID 4
[DEBUG] Nouvelle valeur mappée : '1980’s' -> ID 5
[DEBUG] Nouvelle valeur mappée : '1970’s ou avant' -> ID 6
[DEBUG] Nouvelle valeur mappée : 'musique classique' -> ID 7
[DEBUG] Nouvelle valeur mappée : 'Ne souhaite pas répondre' -> ID 8


{'2020’s': 1,
 '2010’s': 2,
 '2000’s': 3,
 '1990’s': 4,
 '1980’s': 5,
 '1970’s ou avant': 6,
 'musique classique': 7,
 'Ne souhaite pas répondre': 8}

In [40]:

#--------------clean colonne------------
colonnes = ["Quels genres écoutez-vous le plus ?",
 "Période(s) écoutée(s) ",
 "Langue(s) écoutée(s)",
 "Sur quelle plateforme écoutez vous de la musique ?",
 "À quelle fréquence écoutez-vous de la musique ?",
 "Souhaitez-vous répondre à des questions plus personnelles ?",
 "Quel est votre âge ?",
 "Sous quel(s) mood(s) écoutez-vous de la musique ?",
 "Quelle est la durée moyenne des sons que vous écoutez ?",
 "Dans quel contexte écoutez-vous vos musiques ?",
 "Quel est votre artiste ou groupe préféré ?",
 "Quel est votre morceau préféré en ce moment ? ",
 "La musique influence-t-elle votre humeur ?",
 "Quelle est votre situation  ? ",
 "Quel est votre sexe ?"]




for col in colonnes:
    col_nom = safe_filename(str(col))

    nom_fichier_tmp = f"./table/{col_nom}_temp.csv"
    nom_fichier = f"./table/{col_nom}.csv"
    
    keep_columns("./output/output.csv", nom_fichier_tmp, [col])
    
    filter(
        nom_fichier_tmp,
        nom_fichier,
        col,
        ["Ne souhaite pas répondre"]
    )
    os.remove(nom_fichier_tmp)



In [41]:
for col in colonnes:
    col_nom = safe_filename(str(col))

    nom_fichier_tmp = f"./relation/{col_nom}_temp.csv"
    nom_fichier = f"./relation/{col_nom}.csv"
    
    keep_columns("./output/output.csv", nom_fichier_tmp, [col])
    
    filter(
        nom_fichier_tmp,
        nom_fichier,
        col,
        ["Ne souhaite pas répondre"]
    )
    os.remove(nom_fichier_tmp)

In [42]:
colonnes_dict = {"Quels genres écoutez-vous le plus ?":map.map_genres_complets,
 "Période(s) écoutée(s) ":map.map_periode,
 "Langue(s) écoutée(s)":map.map_langue_iso,
 "Sur quelle plateforme écoutez vous de la musique ?":map.map_platforme,
 "À quelle fréquence écoutez-vous de la musique ?":map.map_frequence,
 "Souhaitez-vous répondre à des questions plus personnelles ?":map.map_question_perso,
 "Quel est votre âge ?":map.map_age,
 "Sous quel(s) mood(s) écoutez-vous de la musique ?":map.map_mood,
 "Quelle est la durée moyenne des sons que vous écoutez ?":map.map_durée,
 "Dans quel contexte écoutez-vous vos musiques ?":map.map_contexte,
 "Quel est votre artiste ou groupe préféré ?":map.map_artist_full,
 "Quel est votre morceau préféré en ce moment ? ":map.map_chanson_full,
 "La musique influence-t-elle votre humeur ?":map.map_O_N_humeur,
 "Quelle est votre situation  ? ":map.map_situation,
 "Quel est votre sexe ?":map.map_sexe}




for col in colonnes:
    col_nom = safe_filename(str(col))

    donne_table_association(input='./input.csv',colonne=col,)


[DEBUG] Colonne détectée : 'Quels genres écoutez-vous le plus ?'
[DEBUG] Nouvelle valeur mappée : 'Electro / Techno' -> ID 1
[DEBUG] Nouvelle valeur mappée : 'Pop' -> ID 2
[DEBUG] Nouvelle valeur mappée : 'Rap' -> ID 3
[DEBUG] Nouvelle valeur mappée : "R'n'B" -> ID 4
[DEBUG] Nouvelle valeur mappée : 'Musique latine' -> ID 5
[DEBUG] Nouvelle valeur mappée : 'Disco' -> ID 6
[DEBUG] Nouvelle valeur mappée : 'K-pop' -> ID 7
[DEBUG] Nouvelle valeur mappée : 'Reggae' -> ID 8
[DEBUG] Nouvelle valeur mappée : "Rock 'n' roll" -> ID 9
[DEBUG] Nouvelle valeur mappée : 'Hip-Hop' -> ID 10
[DEBUG] Nouvelle valeur mappée : 'Classique' -> ID 11
[DEBUG] Nouvelle valeur mappée : 'Hard rock' -> ID 12
[DEBUG] Nouvelle valeur mappée : 'Metal' -> ID 13
[DEBUG] Nouvelle valeur mappée : 'Funk' -> ID 14
[DEBUG] Nouvelle valeur mappée : "musique d'avertissement de la SNCF" -> ID 15
[DEBUG] Nouvelle valeur mappée : 'Country' -> ID 16
[DEBUG] Nouvelle valeur mappée : 'Punk' -> ID 17
[DEBUG] Nouvelle valeur mappée

In [43]:
colonnes = ["Quels genres écoutez-vous le plus ?",
 "Période(s) écoutée(s) ",
 "Langue(s) écoutée(s)",
 "Sur quelle plateforme écoutez vous de la musique ?",
 "À quelle fréquence écoutez-vous de la musique ?",
 "Souhaitez-vous répondre à des questions plus personnelles ?",
 "Quel est votre âge ?",
 "Sous quel(s) mood(s) écoutez-vous de la musique ?",
 "Quelle est la durée moyenne des sons que vous écoutez ?",
 "Dans quel contexte écoutez-vous vos musiques ?",
 "Quel est votre artiste ou groupe préféré ?",
 "Quel est votre morceau préféré en ce moment ? ",
 "La musique influence-t-elle votre humeur ?",
 "Quelle est votre situation  ? ",
 "Quel est votre sexe ?"]

map_ = donne_table_association(input='./clean/clean.csv',colonne= "Sur quelle plateforme écoutez vous de la musique ?")

print("map : \n", map_)


map_genre = {
    'Electro / Techno': 1, 
    'Pop': 2, 
    'Rap': 3, 
    "R'n'B": 4, 
    'Musique latine': 5, 
    'Disco': 6, 
    'K-pop': 7, 
    'Reggae': 8, 
    "Rock 'n' roll": 9, 
    'Hip-Hop': 10, 
    'Classique': 11, 
    'Hard rock': 12, 
    'Metal': 13, 
    'Funk': 14, 
    'Country': 15, 
    'Punk': 16, 
    'Blues': 17,
    'Musique bretonne': 18, 
    'Jazz': 19, 
    'Shatta': 20, 
    'Synthwave': 21
            }

map_periode = {
    '2020’s': 1, 
    '2010’s': 2, 
    '2000’s': 3, 
    '1990’s': 4, 
    '1980’s': 5, 
    '1970’s ou avant': 6
    }

map_langue = {
    'Français': 1, 
    'Anglais': 2, 
    'Portugais': 3, 
    'Espagnol': 4, 
    'allemand': 5, 
    'Coréen': 6, 
    'Sans parole': 7, 
    'Indien': 8, 
    'Japonais': 9, 
    'Allemand': 10, 
    'Cyrillique': 11, 
    'Chinois': 12, 
    'Italien': 13, 
    'Russe': 14, 
    'Arabe': 15
    }

map_platforme = {
    'Spotify': 1, 
    'AppleMusic': 2, 
    'Youtube': 3, 
    'Radio': 4, 
    'Deezer': 5, 
    'Soundcloud': 6, 
    'TV': 7, 
    'Téléchargée': 8,
    'CD': 9, 
    'Amerigo': 10, 
    'Amazon music': 11, 
    'Nintendo Music': 12, 
    'FreeMusic': 13
    }


map_frequence = {
    'Plus d’une fois par jour': 1, 
    'Plus d’une fois par semaine': 2, 
    "plus d'une fois par heure": 3,
    "Moins d'une fois par mois": 4, 
    'Plus d’une fois par mois': 5
    }

map_question_perso ={
                    'Oui': 1,
                    'Non': 2
                    }

map_age = {
            '20 - 29 ans': 1,
            '10 - 19 ans': 2,
            '40 - 49 ans': 3,
            '50+ ans': 4,
            '30 - 39 ans': 5
            }

map_mood = {
    'Good vibes': 1, 
    'Soirée': 2, 
    'Chill': 3, 
    'Love': 4, 
    'Motivation': 5, 
    'Tristesse': 6, 
    'Focus': 7
    }

map_durée = {
    'Moins 5 minutes': 1, 
    'Plus de 10 minutes': 2, 
    'Moins de 10 minutes': 3, 
    'Moins de 2 minutes': 4
    }

map_contexte = {
    'Entre amis': 1, 
    'En famille': 2, 
    'Seul(e)': 3, 
    'En fête': 4, 
    'Avec ma copine': 5, 
    'Sur la route': 6, 
    'Au travail': 7, 
    'Sport': 8
    }

map_artiste = {
    'Damso': 1, 
    'Lana Del Rey': 2, 
    'Ninho': 3, 
    'Tiakola': 4, 
    'Fantasm': 5, 
    'Restricted': 6, 
    'Jean Jacques Goldman': 7, 
    'Stray Kids': 8, 
    'Mili': 9, 
    'Imagine dragons': 10, 
    'Ziak': 11, 
    'Josman': 12, 
    'Laylow': 13, 
    'Yoasobi': 14, 
    'Joji': 15, 
    'System of a Down': 16, 
    'Skillet': 17, 
    'Luther': 18, 
    'Renaud': 19, 
    'Livaï': 20, 
    'Three Days Grace': 21, 
    'Billie Eilish': 22, 
    'Queen': 23, 
    'Calogero': 24, 
    'Femtogo': 25, 
    'Bad Bunny': 26,
    'Stromae': 27, 
    'Muse': 28, 
    'Blackpink': 29, 
    'Barbara Pravi': 30, 
    'Kyo': 31, 
    'Radiohead': 32, 
    'Ed sheran': 33, 
    "Diam's": 34, 
    'Dropkick Murphys': 35, 
    'Azali': 36, 
    'Q-Lee': 37, 
    'bbno$': 38, 
    'Creeds': 39, 
    'accept': 40, 
    'Galneryus': 41, 
    'Motorhead': 42, 
    'Van Halen': 43, 
    'Jorge Rivera-Herrans': 44, 
    'Linkin Park': 45, 
    'Poppy': 46, 
    'Coelho': 47, 
    'La mano': 48, 
    'Gims': 49, 
    'Mathieu shedid': 50,
    'Sepultura': 51, 
    'MIGOS': 52, 
    'U2': 53, 
    'Gorillaz': 54, 
    'Gloryhammer': 55, 
    'Against the curent': 56, 
    'Dua lipa': 57, 
    'Depeche mode': 58, 
    'Bennykay': 59, 
    'Vilk': 60, 
    'Powerwolf': 61, 
    'Indochine': 62, 
    'The cure': 63, 
    'Lorenzo': 64, 
    'Fredz': 65, 
    'Jorja smith': 66, 
    'Oliver Tree': 67, 
    'seventeen': 68, 
    'theodora': 69, 
    'plk': 70, 
    'The weeknd': 71, 
    'Coldplay': 72, 
    'Offpring': 73, 
    'Zola': 74, 
    'Artic monkeys': 75, 
    'Bad bunny': 76, 
    'Wallace cleaver': 77, 
    'Catherine ringer': 78, 
    'Fka twigs': 79, 
    'Keith jarett': 80, 
    'Guetta': 81, 
    'OxT': 82, 
    'L’entourloupe': 83, 
    'daft punks': 84, 
    'Yeat': 85
    }

map_chanson = {
    'Un monde à l’autre': 1, 
    'Parfum Dakeez': 2, 
    'Chaque jour': 3, 
    'Buttercup with an Axe': 4, 
    'Puisque tu pars': 5, 
    'Dancing queen': 6, 
    'World.execute(me)': 7, 
    'Bleu soleil luiza': 8, 
    '112': 9, 
    'My heart.': 10, 
    'Starlight': 11, 
    'Baja': 12, 
    'Oblivion': 13, 
    'Diamond Eyes': 14, 
    'melodrama': 15, 
    'Charger Remix 70': 16, 
    "Nous deux c'est mieux": 17, 
    'Riptide': 18, 
    'the rain formerly known as purple': 19, 
    'Just Keep watching': 20, 
    'Barcola': 21, 
    'Les zhommes': 22, 
    'Red Bottoms': 23, 
    'DANCE CRIP': 24, 
    'Music Sounds Better With You': 25, 
    'New Born': 26, 
    'A Song for Us to Sing': 27, 
    'Remember Me': 28, 
    '0gravité': 29, 
    'Soleil bleu': 30, 
    'Na Le Omiki': 31, 
    'dans mon élément': 32, 
    'Indigo night': 33, 
    'On my own': 34, 
    'El biper': 35, 
    'Afsana': 36, 
    'Ordinary': 37, 
    'Dynasties & Dystopia': 38, 
    'fast as a shark': 39,
    'Swimming Pools': 40, 
    'Mesh': 41, 
    'Enter Sandman': 42, 
    'Surviving on Defiance': 43, 
    'Valhalla Calling Again: Dark Viking Music & Valkyrie Battle Songs (Dark Ritual Folk)': 44, 
    'I’am sorry': 45, 
    'remember the time': 46, 
    'ciel': 47, 
    'Salop.e': 48, 
    'la jeune fille du metro': 49, 
    'ratamahatta': 50, 
    'Funny': 51,
    'Comme Benze': 52, 
    'Big jet plane': 53, 
    'Kingslayer': 54, 
    'Unravelling': 55, 
    'grace': 56, 
    'September': 57, 
    'Dead dance': 58, 
    'Ne saute pas': 59, 
    'Oxygene': 60, 
    'Your better half': 61, 
    'two steps from hell': 62, 
    'Lumière': 63, 
    'Impardonnable': 64, 
    'Summer body': 65, 
    'child in time': 66, 
    'un monde à l’autre': 67, 
    'Gold guns girl': 68, 
    'Fête des années 70': 69, 
    'come undone': 70, 
    'New estate': 71, 
    'Птица': 72, 
    'fashion': 73, 
    'Zou bisou': 74, 
    'Golden': 75, 
    'Forgotten Words': 76, 
    'Trop beau': 77, 
    'Obligé de prendre': 78,
    'Hoolywoo': 79, 
    'Policeman': 80, 
    'Dtmf': 81, 
    'g.lasée': 82, 
    'Babymoon - Sokuu': 83, 
    'One love': 84, 
    'Tequila coma': 85, 
    'Indifférence': 86,
    "Un monde à l'autre": 87, 
    'Comfortably numb': 88, 
    'Gold': 89, 'Neverending Night': 90, 
    'L’entourloupe': 91, 
    'memory reboot': 92, 
    'Dernier bolide': 93
    }


map_O_N_humeur = {
                'Non': 1, 
                'Oui': 2
                }

map_situation = {
    'Etudiant(e)': 1, 
    'Artisan': 2, 
    'En emploi': 3, 
    "En recherche d'emploi": 4, 
    'Retraité(e)': 5
    }


map_sexe = {
    'Homme': 1, 
    'Femme': 2
    }






[DEBUG] Colonne détectée : 'Sur quelle plateforme écoutez vous de la musique ?'
[DEBUG] Nouvelle valeur mappée : 'Spotify' -> ID 1
[DEBUG] Nouvelle valeur mappée : 'AppleMusic' -> ID 2
[DEBUG] Nouvelle valeur mappée : 'Youtube' -> ID 3
[DEBUG] Nouvelle valeur mappée : 'Radio' -> ID 4
[DEBUG] Nouvelle valeur mappée : 'Deezer' -> ID 5
[DEBUG] Nouvelle valeur mappée : 'Soundcloud' -> ID 6
[DEBUG] Nouvelle valeur mappée : 'TV' -> ID 7
[DEBUG] Nouvelle valeur mappée : 'Téléchargée' -> ID 8
[DEBUG] Nouvelle valeur mappée : 'CD' -> ID 9
[DEBUG] Nouvelle valeur mappée : 'Amerigo' -> ID 10
[DEBUG] Nouvelle valeur mappée : 'Amazon music' -> ID 11
[DEBUG] Nouvelle valeur mappée : 'Nintendo Music' -> ID 12
[DEBUG] Nouvelle valeur mappée : 'FreeMusic' -> ID 13
map : 
 {'Spotify': 1, 'AppleMusic': 2, 'Youtube': 3, 'Radio': 4, 'Deezer': 5, 'Soundcloud': 6, 'TV': 7, 'Téléchargée': 8, 'CD': 9, 'Amerigo': 10, 'Amazon music': 11, 'Nintendo Music': 12, 'FreeMusic': 13}


In [44]:
map_periode = {
    '2020’s': 1, 
    '2010’s': 2, 
    '2000’s': 3, 
    '1990’s': 4, 
    '1980’s': 5, 
    '1970’s ou avant': 6
    }

# print(f"map_periode {len(map_periode.keys())}")

t = donne_table_association(input='./clean/clean.csv',colonne="Période(s) écoutée(s) ",mapping_dict_=map_periode,only_map=True)
print(t)

[DEBUG] Colonne détectée : 'Période(s) écoutée(s)'
[]
//id_Période(s) écoutée(s)//
{'2020’s': 1, '2010’s': 2, '2000’s': 3, '1990’s': 4, '1980’s': 5, '1970’s ou avant': 6}


In [45]:
m = clean.remapper_colonne_csv('./clean/clean.csv','./final/final.csv','Langue(s) écoutée(s)' , map.map_langue_to_iso)
t = clean.donne_table_association(input='./final/final.csv',colonne="Langue(s) écoutée(s)",mapping_dict_=map.map_langue_iso)
print(t)
print(m)

[DEBUG] Colonne détectée : 'Langue(s) écoutée(s)'
[DEBUG] Nouvelle valeur mappée : 'allemand' -> ID 47
[DEBUG] Nouvelle valeur mappée : 'None' -> ID 48
{'en': 1, 'es': 2, 'fi': 3, 'pt': 4, 'tr': 5, 'sw': 6, 'el': 7, 'ar': 8, 'pl': 9, 'id': 10, 'tw': 11, 'eu': 12, 'ms': 13, 'fr': 14, 'ty': 15, 'hy': 16, 'hi': 17, 'vi': 18, 'ja': 19, 'tl': 20, 'it': 21, 'my': 22, 'gu': 23, 'zh': 24, 'uk': 25, 'az': 26, 'sr': 27, 'lt': 28, 'th': 29, 'bg': 30, 'de': 31, 'no': 32, 'ko': 33, 'uz': 34, 'ka': 35, 'ha': 36, 'sk': 37, 'nl': 38, 'bm': 39, 'fa': 40, 'he': 41, 'ru': 42, 'sv': 43, 'cs': 44, 'la': 45, 'ee': 46, 'allemand': 47, 'None': 48}
None


# reclamation yan

## un csv user_artist_favorite avec les colonnes user_id et artist_id
## un csv track_user_favorite avec les colonnes user_id et track_id

In [ ]:
# #Quel est votre artiste ou groupe préféré ?,Quel est votre morceau préféré en ce moment ?

# m = clean.remapper_colonne_csv('./clean/clean.csv','./final/final.csv','Quel est votre artiste ou groupe préféré ?' , map.map_langue_to_iso)
t = clean.donne_table_association(input='./final/final.csv',colonne='Quel est votre artiste ou groupe préféré ?',mapping_dict_=map.map_artist_csvprof)
print(t)
clean.dict_to_csv(map.map_artist_full,"./final/artist_yd.csv",'artist_nom','artist_id')

[DEBUG] Colonne détectée : 'Quel est votre artiste ou groupe préféré ?'
[DEBUG] Nouvelle valeur mappée : 'Damso' -> ID 24358
[DEBUG] Nouvelle valeur mappée : 'Lana Del Rey' -> ID 24359
[DEBUG] Nouvelle valeur mappée : 'Ninho' -> ID 24360
[DEBUG] Nouvelle valeur mappée : 'Tiakola' -> ID 24361
[DEBUG] Nouvelle valeur mappée : 'Fantasm' -> ID 24362
[DEBUG] Nouvelle valeur mappée : 'Restricted' -> ID 24363
[DEBUG] Nouvelle valeur mappée : 'Jean Jacques Goldman' -> ID 24364
[DEBUG] Nouvelle valeur mappée : 'Stray Kids' -> ID 24365
[DEBUG] Nouvelle valeur mappée : 'Mili' -> ID 24366
[DEBUG] Nouvelle valeur mappée : 'Imagine dragons' -> ID 24367
[DEBUG] Nouvelle valeur mappée : 'Ziak' -> ID 24368
[DEBUG] Nouvelle valeur mappée : 'Josman' -> ID 24369
[DEBUG] Nouvelle valeur mappée : 'Laylow' -> ID 24370
[DEBUG] Nouvelle valeur mappée : 'Yoasobi' -> ID 24371
[DEBUG] Nouvelle valeur mappée : 'Joji' -> ID 24372
[DEBUG] Nouvelle valeur mappée : 'System of a Down' -> ID 24373
[DEBUG] Nouvelle valeu

In [ ]:
t = clean.donne_table_association(input='./final/final.csv',colonne="Quel est votre morceau préféré en ce moment ? ",mapping_dict_=map.map_chanson_csvprof)
print(t)
clean.dict_to_csv(map.map_chanson_full,"./final/track_id.csv",'chanson_titre','chanson_id')
# clean.

[DEBUG] Colonne détectée : 'Quel est votre morceau préféré en ce moment ?'
[DEBUG] Nouvelle valeur mappée : 'Un monde à l’autre' -> ID 155321
[DEBUG] Nouvelle valeur mappée : 'Parfum Dakeez' -> ID 155322
[DEBUG] Nouvelle valeur mappée : 'Chaque jour' -> ID 155323
[DEBUG] Nouvelle valeur mappée : 'Buttercup with an Axe' -> ID 155324
[DEBUG] Nouvelle valeur mappée : 'Puisque tu pars' -> ID 155325
[DEBUG] Nouvelle valeur mappée : 'Dancing queen' -> ID 155326
[DEBUG] Nouvelle valeur mappée : 'World.execute(me)' -> ID 155327
[DEBUG] Nouvelle valeur mappée : 'Bleu soleil luiza' -> ID 155328
[DEBUG] Nouvelle valeur mappée : '112' -> ID 155329
[DEBUG] Nouvelle valeur mappée : 'My heart.' -> ID 155330
[DEBUG] Nouvelle valeur mappée : 'Baja' -> ID 155331
[DEBUG] Nouvelle valeur mappée : 'Diamond Eyes' -> ID 155332
[DEBUG] Nouvelle valeur mappée : 'melodrama' -> ID 155333
[DEBUG] Nouvelle valeur mappée : 'Charger Remix 70' -> ID 155334
[DEBUG] Nouvelle valeur mappée : "Nous deux c'est mieux" -> I